In [1]:
import pandas as pd
import numpy as np
import MetaTrader5 as mt5
from datetime import datetime, timedelta
import os
import glob
import warnings
import ta
import logging
import matplotlib.pyplot as plt
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Spacer
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.units import inch
import locale

warnings.filterwarnings('ignore')

# Configurar localização para formato brasileiro
try:
    locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
except locale.Error:
    try:
        locale.setlocale(locale.LC_ALL, 'Portuguese_Brazil.1252')
    except locale.Error:
        print("Aviso: Não foi possível configurar localização brasileira")

def format_currency_br(value):
    """Formata números no padrão brasileiro: 5.688,00"""
    if pd.isna(value):
        return "N/A"
    
    # Converte para string com 2 casas decimais
    formatted = f"{value:.2f}"
    
    # Separa parte inteira e decimal
    parts = formatted.split('.')
    integer_part = parts[0]
    decimal_part = parts[1]
    
    # Adiciona pontos para milhares na parte inteira
    if len(integer_part) > 3:
        # Inverte a string para adicionar pontos de trás para frente
        reversed_int = integer_part[::-1]
        formatted_int = '.'.join([reversed_int[i:i+3] for i in range(0, len(reversed_int), 3)])
        integer_part = formatted_int[::-1]
    
    return f"{integer_part},{decimal_part}"

def format_volume_br(value):
    """Formata volume no padrão brasileiro"""
    if pd.isna(value):
        return "N/A"
    
    # Para valores inteiros (volume)
    formatted = f"{int(value)}"
    
    if len(formatted) > 3:
        reversed_int = formatted[::-1]
        formatted_int = '.'.join([reversed_int[i:i+3] for i in range(0, len(reversed_int), 3)])
        formatted = formatted_int[::-1]
    
    return formatted

DATA_DIR = "C://Users//thiag//OneDrive//ARQUIVOS//Bolsa//COLETA//MT//FUTUROS"

TIMEFRAMES = {
    'M5': mt5.TIMEFRAME_M5,
    'M15': mt5.TIMEFRAME_M15,
    'H1': mt5.TIMEFRAME_H1,
    'M2': mt5.TIMEFRAME_M2,
    'D1': mt5.TIMEFRAME_D1,
    'W1': mt5.TIMEFRAME_W1,
    'MN1': mt5.TIMEFRAME_MN1
}

class Config:
    BASE_DIR = "C://Users//thiag//OneDrive//ARQUIVOS//Bolsa//COLETA//MT//FUTUROS"
    CACHE_DIR = os.path.join(BASE_DIR, "cache")
    MIN_VOLUME = 2000
    TOP_LEVELS = 10
    PRICE_RANGE = 50
    REPORT_FILE = "Report_v15.pdf"
    LOG_FILE = "market_analyzer.log"

    def __init__(self):
        os.makedirs(self.CACHE_DIR, exist_ok=True)

class MTDataCollector:
    def __init__(self):
        if not mt5.initialize():
            print("MetaTrader5 não iniciou, erro:", mt5.last_error())
            raise Exception("Falha na inicialização do MetaTrader5")
        
        os.makedirs(DATA_DIR, exist_ok=True)
        self.existing_files = self.get_existing_files()
    
    def __del__(self):
        mt5.shutdown()

    def get_existing_files(self):
        files = glob.glob(os.path.join(DATA_DIR, 'WDO*.csv'))
        return {os.path.basename(f) for f in files}

    def get_timeframe_data(self, symbol, timeframe, date):
        start_time = datetime(date.year, date.month, date.day, 6, 0)
        end_time = datetime(date.year, date.month, date.day, 21, 00)
        
        try:
            rates = mt5.copy_rates_range(symbol, timeframe, start_time, end_time)
            if rates is None or len(rates) == 0:
                return None
                
            df = pd.DataFrame(rates)
            df['time'] = pd.to_datetime(df['time'], unit='s')
            return df
            
        except Exception as e:
            print(f"Erro ao coletar {symbol} em {timeframe} para {date.date()}: {str(e)}")
            return None

    def process_data(self, df):
        if df is None or len(df) == 0:
            return None
            
        df = df.copy()
        
        df['range'] = df['high'] - df['low']
        df['body'] = abs(df['close'] - df['open'])
        df['upper_wick'] = df['high'] - df[['open', 'close']].max(axis=1)
        df['lower_wick'] = df[['open', 'close']].min(axis=1) - df['low']
        
        df['volume_ema'] = df['tick_volume'].ewm(span=20).mean()
        df['volume_ratio'] = df['tick_volume'] / df['volume_ema']
        
        df['price_change'] = df['close'].pct_change()
        df['price_change_pct'] = df['price_change'] * 100
        
        for period in [8, 20, 50]:
            df[f'ma_{period}'] = df['close'].rolling(window=period).mean()
        
        df['volatility'] = df['range'].rolling(window=20).std()
        df['atr'] = df['range'].rolling(window=14).mean()
        
        df['trend'] = np.where(df['ma_20'] > df['ma_50'], 1, 
                             np.where(df['ma_20'] < df['ma_50'], -1, 0))
        
        return df

    def get_filename(self, symbol, timeframe, date):
        tf_name = {v: k for k, v in TIMEFRAMES.items()}[timeframe]
        return f"{symbol.replace('$', '')}_{tf_name}_{date.strftime('%Y%m%d')}.csv"

    def save_data(self, df, symbol, timeframe, date):
        if df is None:
            return None
            
        filename = self.get_filename(symbol, timeframe, date)
        filepath = os.path.join(DATA_DIR, filename)
        
        column_mapping = {
            'open': 'PREÇO ABERT.',
            'high': 'PREÇO MÁX.',
            'low': 'PREÇO MÍN.',
            'close': 'ÚLT. PREÇO',
            'tick_volume': 'VOL.',
            'time': 'DATA'
        }
        
        df_save = df.rename(columns=column_mapping)
        df_save['AJUSTE'] = df_save['ÚLT. PREÇO']
        df_save['VENCTO'] = symbol.split('$')[0]
        df_save['TIMEFRAME'] = {v: k for k, v in TIMEFRAMES.items()}[timeframe]
        
        primary_columns = ['VENCTO', 'DATA', 'TIMEFRAME', 'PREÇO ABERT.', 'PREÇO MÍN.', 
                          'PREÇO MÁX.', 'ÚLT. PREÇO', 'AJUSTE', 'VOL.']
        other_columns = [col for col in df_save.columns if col not in primary_columns]
        df_save = df_save[primary_columns + other_columns]
        
        df_save.to_csv(filepath, index=False)
        return filepath

    def collect_historical_data(self, days=30):
        symbols = ['WDO$N']
        collected_files = {}
        
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days)
        current_date = start_date
        
        print(f"\nIniciando coleta de {days} dias: {start_date.date()} até {end_date.date()}")
        print("="*70)
        
        while current_date <= end_date:
            print(f"\nProcessando data: {current_date.date()}")
            
            for symbol in symbols:
                for tf_name, timeframe in TIMEFRAMES.items():
                    filename = self.get_filename(symbol, timeframe, current_date)
                    
                    if filename in self.existing_files:
                        print(f"  {symbol} {tf_name}: Arquivo já existe")
                        continue
                    
                    try:
                        print(f"  Coletando {symbol} {tf_name}...", end=' ')
                        
                        df = self.get_timeframe_data(symbol, timeframe, current_date)
                        if df is None:
                            print("Sem dados")
                            continue
                            
                        df_processed = self.process_data(df)
                        if df_processed is None:
                            print("Erro no processamento")
                            continue
                            
                        filepath = self.save_data(df_processed, symbol, timeframe, current_date)
                        if filepath:
                            key = f"{symbol}_{tf_name}_{current_date.date()}"
                            collected_files[key] = {
                                'path': filepath,
                                'records': len(df_processed)
                            }
                            print(f"OK ({len(df_processed)} registros)")
                            self.existing_files.add(filename)
                        
                    except Exception as e:
                        print(f"Erro: {str(e)}")
                        continue
            
            current_date += timedelta(days=1)
        
        return collected_files

class MarketAnalyzer:
    def __init__(self, config):
        self.config = config
        self.setup_logging()
        
    def setup_logging(self):
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler(os.path.join(self.config.CACHE_DIR, self.config.LOG_FILE)),
                logging.StreamHandler()
            ]
        )
        self.logger = logging.getLogger(__name__)

    def load_data(self):
        all_data = []
        for filename in os.listdir(self.config.BASE_DIR):
            if filename.startswith("WDO") and filename.endswith(".csv"):
                file_path = os.path.join(self.config.BASE_DIR, filename)
                df = pd.read_csv(file_path)
                df['DATA'] = pd.to_datetime(df['DATA'])
                all_data.append(df)
        
        if not all_data:
            self.logger.error("Nenhum arquivo WDO encontrado no diretório.")
            return None
        
        combined_df = pd.concat(all_data, ignore_index=True)
        combined_df = combined_df.sort_values('DATA')
        return combined_df

    def add_technical_indicators(self, df):
        df = df.copy()
        df['sma_20'] = ta.trend.sma_indicator(df['ÚLT. PREÇO'], window=20)
        df['ema_20'] = ta.trend.ema_indicator(df['ÚLT. PREÇO'], window=20)
        df['rsi'] = ta.momentum.rsi(df['ÚLT. PREÇO'], window=14)
        df['stoch_k'] = ta.momentum.stoch(df['PREÇO MÁX.'], df['PREÇO MÍN.'], df['ÚLT. PREÇO'], window=14)
        df['stoch_d'] = ta.momentum.stoch_signal(df['PREÇO MÁX.'], df['PREÇO MÍN.'], df['ÚLT. PREÇO'], window=14)
        df['atr'] = ta.volatility.average_true_range(df['PREÇO MÁX.'], df['PREÇO MÍN.'], df['ÚLT. PREÇO'], window=14)
        return df

    def identify_levels(self, df, last_close):
        supports = {}
        resistances = {}
        
        for _, row in df.iterrows():
            if last_close - self.config.PRICE_RANGE <= row['PREÇO MÍN.'] < last_close and row['VOL.'] > self.config.MIN_VOLUME:
                price = row['PREÇO MÍN.']
                if price not in supports:
                    supports[price] = {
                        "price": price,
                        "volume": row['VOL.'],
                        "date": row['DATA'],
                        "strength": self.calculate_strength(row['VOL.'], row['DATA'], df, 1),
                        "touches": 1
                    }
                else:
                    supports[price]["volume"] += row['VOL.']
                    supports[price]["touches"] += 1
                    supports[price]["date"] = max(supports[price]["date"], row['DATA'])
                    supports[price]["strength"] = self.calculate_strength(
                        supports[price]["volume"], 
                        supports[price]["date"], 
                        df, 
                        supports[price]["touches"]
                    )
            
            if last_close < row['PREÇO MÁX.'] <= last_close + self.config.PRICE_RANGE and row['VOL.'] > self.config.MIN_VOLUME:
                price = row['PREÇO MÁX.']
                if price not in resistances:
                    resistances[price] = {
                        "price": price,
                        "volume": row['VOL.'],
                        "date": row['DATA'],
                        "strength": self.calculate_strength(row['VOL.'], row['DATA'], df, 1),
                        "touches": 1
                    }
                else:
                    resistances[price]["volume"] += row['VOL.']
                    resistances[price]["touches"] += 1
                    resistances[price]["date"] = max(resistances[price]["date"], row['DATA'])
                    resistances[price]["strength"] = self.calculate_strength(
                        resistances[price]["volume"], 
                        resistances[price]["date"], 
                        df, 
                        resistances[price]["touches"]
                    )
        
        supports = sorted(supports.values(), key=lambda x: x['strength'], reverse=True)[:self.config.TOP_LEVELS]
        resistances = sorted(resistances.values(), key=lambda x: x['strength'], reverse=True)[:self.config.TOP_LEVELS]
        
        return supports, resistances

    def calculate_strength(self, volume, date, df, touches):
        import math
        
        days_diff = (df['DATA'].max() - date).days
        recency_factor = math.exp(-0.1 * days_diff)
        
        max_volume = df['VOL.'].max()
        volume_factor = math.log(1 + volume) / math.log(1 + max_volume) if max_volume > 0 else 0
        
        max_touches = 10
        touches_factor = min(touches / max_touches, 1)
        
        strength = (0.4 * recency_factor + 0.4 * volume_factor + 0.2 * touches_factor)
        
        return strength * 100

    def analyze_market_context(self, df):
        last_price = df['ÚLT. PREÇO'].iloc[-1]
        sma20 = df['sma_20'].iloc[-1]
        
        if last_price > sma20 * 1.02:
            trend = "ALTA ↑"
        elif last_price < sma20 * 0.98:
            trend = "BAIXA ↓"
        else:
            trend = "LATERAL →"
        
        volatility = df['atr'].iloc[-1] / last_price * 100
        avg_volume = df['VOL.'].tail(5).mean()
        
        return {
            'trend': trend,
            'volatility': volatility,
            'avg_volume': avg_volume,
            'rsi': df['rsi'].iloc[-1],
            'stoch_k': df['stoch_k'].iloc[-1],
            'stoch_d': df['stoch_d'].iloc[-1]
        }

    def find_gaps(self, df):
        gaps = []
        for i in range(1, len(df)):
            prev_high = df['PREÇO MÁX.'].iloc[i-1]
            curr_low = df['PREÇO MÍN.'].iloc[i]
            prev_low = df['PREÇO MÍN.'].iloc[i-1]
            curr_high = df['PREÇO MÁX.'].iloc[i]
            
            if curr_low > prev_high:
                gaps.append({
                    'type': 'alta',
                    'size': curr_low - prev_high,
                    'date': df['DATA'].iloc[i],
                    'price': curr_low,
                    'filled': curr_low <= df['PREÇO MÍN.'].iloc[i:].min()
                })
            elif curr_high < prev_low:
                gaps.append({
                    'type': 'baixa',
                    'size': prev_low - curr_high,
                    'date': df['DATA'].iloc[i],
                    'price': curr_high,
                    'filled': curr_high >= df['PREÇO MÁX.'].iloc[i:].max()
                })
        
        return sorted(gaps, key=lambda x: x['size'], reverse=True)[:3]

    def calculate_volume_distribution(self, df):
        last_close = df['ÚLT. PREÇO'].iloc[-1]
        lower_bound = last_close - self.config.PRICE_RANGE
        upper_bound = last_close + self.config.PRICE_RANGE
        
        df['price_range'] = pd.cut(df['ÚLT. PREÇO'], bins=np.arange(lower_bound, upper_bound + 8, 8))
        price_volume = df.groupby('price_range', observed=True)['VOL.'].sum()
        total_volume = price_volume.sum()
        
        volume_dist = [
            {
                'price_range': f"{interval.left:.2f} - {interval.right:.2f}",
                'volume': volume,
                'percentage': (volume / total_volume) * 100
            }
            for interval, volume in price_volume.items()
        ]
        
        volume_dist_sorted = sorted(volume_dist, key=lambda x: x['volume'], reverse=True)
        
        above_close = [level for level in volume_dist_sorted if float(level['price_range'].split(' - ')[0]) >= last_close][:5]
        below_close = [level for level in volume_dist_sorted if float(level['price_range'].split(' - ')[1]) < last_close][:5]
        
        return above_close, below_close

class PDFReportGenerator:
    def __init__(self, config):
        self.config = config
        self.styles = getSampleStyleSheet()
        self.styles.add(ParagraphStyle(name='Small', fontSize=8))

    def generate_report(self, df, supports, resistances, context, gaps, volume_dist):
        all_levels = supports + resistances
        max_touches = max(level['touches'] for level in all_levels)
        min_touches = min(level['touches'] for level in all_levels)
        
        doc = SimpleDocTemplate(
            os.path.join(self.config.CACHE_DIR, self.config.REPORT_FILE),
            pagesize=letter,
            rightMargin=72, leftMargin=72, topMargin=72, bottomMargin=18
        )
        
        elements = []

        elements.append(Paragraph("ANÁLISE MACRO DO WDO", self.styles['Title']))
        elements.append(Paragraph(f"Data/Hora: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}", self.styles['Normal']))
        last_close = df['ÚLT. PREÇO'].iloc[-1]
        elements.append(Paragraph(f"Último Fechamento: {format_currency_br(last_close)}", self.styles['Normal']))
        elements.append(Spacer(1, 0.2*inch))

        elements.append(Paragraph("Distribuição de Volume por Nível:", self.styles['Normal']))
        
        elements.append(Paragraph(f"5 Melhores Níveis acima de {format_currency_br(last_close)}:", self.styles['Small']))
        for dist in volume_dist[0]:
            # Formatar o range de preços
            range_parts = dist['price_range'].split(' - ')
            formatted_range = f"{format_currency_br(float(range_parts[0]))} - {format_currency_br(float(range_parts[1]))}"
            elements.append(Paragraph(
                f"→ {formatted_range}: {format_volume_br(dist['volume'])} ({dist['percentage']:.1f}%)",
                self.styles['Small']
            ))
        
        elements.append(Spacer(1, 0.1*inch))
        
        elements.append(Paragraph(f"5 Melhores Níveis abaixo de {format_currency_br(last_close)}:", self.styles['Small']))
        for dist in volume_dist[1]:
            range_parts = dist['price_range'].split(' - ')
            formatted_range = f"{format_currency_br(float(range_parts[0]))} - {format_currency_br(float(range_parts[1]))}"
            elements.append(Paragraph(
                f"→ {formatted_range}: {format_volume_br(dist['volume'])} ({dist['percentage']:.1f}%)",
                self.styles['Small']
            ))
        
        elements.append(Paragraph(f"NÍVEIS DE RESISTÊNCIA", self.styles['Heading2']))
        for i, level in enumerate(resistances, 1):
            strength_percent = self._create_strength_bar(level, max_touches, min_touches)
            volume_percent = self._create_volume_bar(level['volume'], max(r['volume'] for r in resistances))
            level_text = (
                f"R{i:2d}. Preço: {format_currency_br(level['price'])} | "
                f"Volume: {format_volume_br(level['volume'])} {volume_percent} | "
                f"Força: {strength_percent} | "
                f"Toques: {level['touches']:>2d} | "
                f"Dist: {((level['price'] - last_close) / last_close * 100):>6.2f}% | "
                f"Último: {level['date'].strftime('%Y-%m-%d')} "
            )
            elements.append(Paragraph(level_text, self.styles['Small']))
        elements.append(Spacer(1, 0.2*inch))

        elements.append(Paragraph(f"NÍVEIS DE SUPORTE", self.styles['Heading2']))
        for i, level in enumerate(supports, 1):
            strength_percent = self._create_strength_bar(level, max_touches, min_touches)
            volume_percent = self._create_volume_bar(level['volume'], max(s['volume'] for s in supports))
            level_text = (
                f"S{i:2d}. Preço: {format_currency_br(level['price'])} | "
                f"Volume: {format_volume_br(level['volume'])} {volume_percent} | "
                f"Força: {strength_percent} | "
                f"Toques: {level['touches']:>2d} | "
                f"Dist: {((last_close - level['price']) / last_close * 100):>6.2f}% | "
                f"Último: {level['date'].strftime('%Y-%m-%d')} "
            )
            elements.append(Paragraph(level_text, self.styles['Small']))
        elements.append(Spacer(1, 0.2*inch))

        doc.build(elements)

    def _create_volume_bar(self, volume, max_volume):
        if max_volume <= 0:
            return "[0.0%]"
        
        percentage = (volume / max_volume) * 100
        return f"[{percentage:.1f}%]"
    
    def _create_strength_bar(self, level, max_touches, min_touches):
        touches = level['touches']
        
        if max_touches == min_touches:
            percentage = 100.0
        else:
            percentage = ((touches - min_touches) / (max_touches - min_touches)) * 100
        
        return f"[{percentage:.1f}%]"

def print_collection_summary(collected_files):
    print("\n" + "="*70)
    print("RESUMO DA COLETA DE DADOS")
    print("="*70)
    
    if not collected_files:
        print("\nNenhum novo arquivo coletado - Todos os dados já existem")
        return
    
    for key, info in collected_files.items():
        symbol, tf, date = key.split('_')
        filepath = info['path']
        filename = os.path.basename(filepath)
        filesize = os.path.getsize(filepath) / 1024
        
        print(f"\n{symbol} - {tf} - {date}:")
        print(f"  Arquivo: {filename}")
        print(f"  Tamanho: {filesize:.1f} KB")
        print(f"  Registros: {info['records']}")

def main():
    config = Config()
    analyzer = MarketAnalyzer(config)
    report_generator = PDFReportGenerator(config)

    try:
        df = analyzer.load_data()
        if df is None:
            raise ValueError("Não foi possível carregar os dados")

        df = analyzer.add_technical_indicators(df)
        last_close = df['ÚLT. PREÇO'].iloc[-1]

        supports, resistances = analyzer.identify_levels(df, last_close)
        context = analyzer.analyze_market_context(df)
        gaps = analyzer.find_gaps(df)
        volume_dist_above, volume_dist_below = analyzer.calculate_volume_distribution(df)
    
        report_generator.generate_report(df, supports, resistances, context, gaps, (volume_dist_above, volume_dist_below))

        print(f"Relatório gerado com sucesso: {config.REPORT_FILE}")

    except Exception as e:
        analyzer.logger.error(f"Erro durante a execução: {str(e)}")

if __name__ == "__main__":
    main()

Relatório gerado com sucesso: Report_v15.pdf
